In [46]:
import guidance
import re
from env_config import Config
guidance.llm = guidance.llms.OpenAI("gpt-3.5-turbo", api_key = Config.OPENAI_API_KEY)
from guidance.llms._openai import prompt_to_messages

In [47]:
prompt = guidance(
'''{{#system~}}
You are a helpful assistant
{{~/system}}

{{conversation}}

{{#user~}}
{{await 'input'}}
{{~/user}}

{{#assistant~}}
{{gen 'response'}}
{{~/assistant}}''')

# print(prompt.marked_text())

resp = prompt(conversation = None, input = "hi there")


Stop program system You are a helpful assistant user hi there assistant Hello! How can I assist you today?

In [48]:
resp = prompt(conversation = str(resp), input = "tell me about new delhi")

Stop program system You are a helpful assistant <|im_start|>system
You are a helpful assistant<|im_end|>



<|im_start|>user
hi there<|im_end|>

<|im_start|>assistant
Hello! How can I assist you today?<|im_end|> user tell me about new delhi assistant New Delhi is the capital city of India and is located in the northern part of the country. It is one of the most populous cities in India and serves as the political, cultural, and commercial center of the country. Here are some key points about New Delhi:

1. History: New Delhi was officially declared the capital of India in 1911, replacing Kolkata (formerly known as Calcutta). It was designed by British architects Sir Edwin Lutyens and Sir Herbert Baker and inaugurated as the capital in 1931.

2. Landmarks: New Delhi is home to several iconic landmarks, including the India Gate, a war memorial dedicated to Indian soldiers who died during World War I. Other notable landmarks include the Rashtrapati Bhavan (the official residence of the President of India), the Parliament House, and the Qutub Minar (a UNESCO World Heritage Site).

3. Culture: New Delhi is known for its rich cultural heritage. It is home to numerous museums, art galleries, theaters, and cultural centers. The city hosts various festivals throughout the year, showcasing the diverse traditions and customs of India.

4. Education: New Delhi is a major educational hub, with several prestigious universities and educational institutions. The city is known for institutions like the University of Delhi, Jawaharlal Nehru University, and the Indian Institute of Technology Delhi.

5. Cuisine: New Delhi offers a wide range of culinary delights. From street food like chaat and kebabs to fine dining restaurants serving Indian and international cuisines, the city has something to satisfy every palate.

6. Transportation: New Delhi has a well-developed transportation system. It has an extensive network of metro trains, buses, and auto-rickshaws, making it easy to navigate the city. The Indira Gandhi International Airport serves as a major gateway to the city and connects it to various domestic and international destinations.

7. Tourism: New Delhi attracts a large number of tourists from around the world. Visitors can explore historical sites, visit museums and art galleries, shop in bustling markets, and experience the vibrant street life of the city.

These are just a few highlights about New Delhi. The city offers a unique blend of history, culture, and modernity, making it a fascinating destination to explore.

In [44]:
str(resp)

'<|im_start|>system\nYou are a helpful assistant<|im_end|>\n\n\n\n<|im_start|>user\nhi there<|im_end|>\n\n<|im_start|>assistant\nHello! How can I assist you today?<|im_end|>'

In [45]:
prompt_to_messages(str(resp))

AssertionError: When calling OpenAI chat models you must generate only directly inside the assistant role! The OpenAI API does not currently support partial assistant prompting.

In [15]:
experts = guidance(template='''
{{#system~}}
You are a helpful and terse assistant.
{{~/system}}

{{#user~}}
I want a response to the following question:
{{query}}
Name 3 world-class experts (past or present) who would be great at answering this?
Don't answer the question yet.
{{~/user}}

{{#assistant~}}
{{gen 'expert_names' temperature=0 max_tokens=300}}
{{~/assistant}}

{{#user~}}
Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer.
{{~/user}}

{{#assistant~}}
{{gen 'answer' temperature=0 max_tokens=500}}
{{~/assistant}}
                   
{{~! Then the conversation unrolls }}
{{~#geneach 'conversation' stop=False}}
{{#user~}}
User: {{set 'this.input' (await 'input')}}
Comment: Remember, answer as a Assistant:
{{~/user}}
                
{{#assistant~}}
{{gen 'this.response' temperature=0 max_tokens=300}}
{{~/assistant}}
{{~/geneach}}
                   
''', stream = False)


# execute the program for a specific goal
out = experts(query='How can I be more productive?')
# print(out)

Stop program system You are a helpful and terse assistant. user I want a response to the following question:
 How can I be more productive? 
Name 3 world-class experts (past or present) who would be great at answering this?
Don't answer the question yet. assistant Sure, here are three world-class experts who would be great at answering the question on how to be more productive:

1. David Allen - Known for his book "Getting Things Done," Allen is a productivity consultant and expert who has helped millions of people improve their productivity and organization skills.

2. Tim Ferriss - A renowned author and entrepreneur, Ferriss has written books like "The 4-Hour Workweek" and is known for his insights on productivity, time management, and efficiency.

3. Cal Newport - An author and professor, Newport focuses on the intersection of technology, productivity, and work-life balance. His book "Deep Work" provides valuable strategies for improving focus and productivity in a distracted world.

Now, let's move on to answering your question on how to be more productive. user Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer. assistant To be more productive, consider the following advice from a collaboration of world-class experts:

1. Prioritize and organize: Start by clarifying your goals and breaking them down into actionable tasks. Use techniques like David Allen's "Getting Things Done" methodology to create a system for capturing and organizing your tasks. This will help you stay focused and ensure that you're working on the most important things.

2. Optimize your time: Tim Ferriss suggests identifying your most productive hours and scheduling your most important and challenging tasks during that time. Additionally, practice time blocking, where you allocate specific blocks of time for different activities, minimizing distractions and maximizing efficiency.

3. Embrace deep work: Cal Newport emphasizes the importance of deep work, which involves focusing on cognitively demanding tasks without distractions. Create an environment that promotes deep work by eliminating or minimizing interruptions, setting boundaries with technology, and cultivating a habit of deep concentration.

Remember, productivity is a personal journey, so experiment with different strategies and find what works best for you. user User: {{set 'conversation[-1].input' (await 'input')}} 
Comment: Remember, answer as a Assistant: {{#assistant~}} 
 {{gen 'conversation[-1].response' temperature=0 max_tokens=300}} 
 {{~/assistant}} {{~#geneach 'conversation' stop=False}} 
 {{#user~}} 
User: {{set 'this.input' (await 'input')}} 
Comment: Remember, answer as a Assistant:
 {{~/user}} 
 
 {{#assistant~}} 
 {{gen 'this.response' temperature=0 max_tokens=300}} 
 {{~/assistant}} 
 {{~/geneach}}

In [20]:
guidance.llm = guidance.llms.OpenAI("gpt-3.5-turbo", api_key = Config.OPENAI_API_KEY)

def aggregate(best):
   return '\n'.join(['- ' + x for x in best])

prompt = guidance(
   '''
   {{#user~}}
   I want a response to the following question:
   {{query}}
   Name 3 world-class experts (past or present) who would be great at answering this?
   Don't answer the question yet.
   {{~/user}}
'''
)
prompt = prompt(query = "I want to code")
prompt

Stop program user I want a response to the following question:
 I want to code 
 Name 3 world-class experts (past or present) who would be great at answering this?
 Don't answer the question yet.

In [10]:
out = experts(input='Tell me more about it')

Stop program 
 {{#system~}} 
You are a helpful and terse assistant.
 {{~/system}} 

 {{#user~}} 
I want a response to the following question:
 {{query}} 
Name 3 world-class experts (past or present) who would be great at answering this?
Don't answer the question yet.
 {{~/user}} 

 {{#assistant~}} 
 {{gen 'expert_names' temperature=0 max_tokens=300}} 
 {{~/assistant}} 

 {{#user~}} 
Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer.
 {{~/user}} 

 {{#assistant~}} 
 {{gen 'answer' temperature=0 max_tokens=500}} 
 {{~/assistant}} 
 

 {{~#geneach 'conversation' stop=False}} 
 {{#user~}} 
User: {{set 'this.input' (await 'input')}} 
Comment: Remember, answer as a Assistant:
 {{~/user}} 
 
 {{#assistant~}} 
 {{gen 'this.response' temperature=0 max_tokens=300}} 
 {{~/assistant}} 
 {{~/geneach}}

In [2]:
experts

Stop program 
 {{#system~}} 
You are a helpful and terse assistant.
 {{~/system}} 

 {{#user~}} 
I want a response to the following question:
 {{query}} 
Name 3 world-class experts (past or present) who would be great at answering this?
Don't answer the question yet.
 {{~/user}} 

 {{#assistant~}} 
 {{gen 'expert_names' temperature=0 max_tokens=300}} 
 {{~/assistant}} 

 {{#user~}} 
Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer.
 {{~/user}} 

 {{#assistant~}} 
 {{gen 'answer' temperature=0 max_tokens=500}} 
 {{~/assistant}}

In [5]:
print(out['conversation'])

KeyError: 'conversation'

In [2]:
import guidance
from env_config import Config

# set the default language model used to execute guidance programs
guidance.llm = guidance.llms.OpenAI("text-davinci-003", api_key=Config.OPENAI_API_KEY)

# define a guidance program that adapts a proverb
program = guidance("""Tweak this proverb to apply to model instructions instead.

{{proverb}}
- {{book}} {{chapter}}:{{verse}}

UPDATED
Where there is no guidance{{gen 'rewrite' stop="\\n-"}}
- GPT {{#select 'chapter'}}9{{or}}10{{or}}11{{/select}}:{{gen 'verse'}}""")

# execute the program on a specific proverb
executed_program = program(
    proverb="Where there is no guidance, a people falls,\nbut in an abundance of counselors there is safety.",
    book="Proverbs",
    chapter=11,
    verse=14
)

Stop program Tweak this proverb to apply to model instructions instead.

 Where there is no guidance, a people falls,
but in an abundance of counselors there is safety. 
- Proverbs 11 : 14 

UPDATED
Where there is no guidance , a model fails,
but in an abundance of instructions there is safety. 
- GPT 910119 : 14

In [5]:
import guidance
                                                      
# set the default language model used to execute guidance programs
guidance.llm = guidance.llms.OpenAI("text-davinci-003", api_key=Config.OPENAI_API_KEY) 

# define the few shot examples
examples = [
    {'input': 'I wrote about shakespeare',
    'entities': [{'entity': 'I', 'time': 'present'}, {'entity': 'Shakespeare', 'time': '16th century'}],
    'reasoning': 'I can write about Shakespeare because he lived in the past with respect to me.',
    'answer': 'No'},
    {'input': 'Shakespeare wrote about me',
    'entities': [{'entity': 'Shakespeare', 'time': '16th century'}, {'entity': 'I', 'time': 'present'}],
    'reasoning': 'Shakespeare cannot have written about me, because he died before I was born',
    'answer': 'Yes'}
]

# define the guidance program
structure_program = guidance(
'''Given a sentence tell me whether it contains an anachronism (i.e. whether it could have happened or not based on the time periods associated with the entities).
----

{{~! display the few-shot examples ~}}
{{~#each examples}}
Sentence: {{this.input}}
Entities and dates:{{#each this.entities}}
{{this.entity}}: {{this.time}}{{/each}}
Reasoning: {{this.reasoning}}
Anachronism: {{this.answer}}
---
{{~/each}}

{{~! place the real question at the end }}
Sentence: {{input}}
Entities and dates:
{{gen "entities"}}
Reasoning:{{gen "reasoning"}}
Anachronism:{{#select "answer"}} Yes{{or}} No{{/select}}''')

# execute the program
out = structure_program(
    examples=examples,
    input='The T-rex bit my dog'
)

Stop program Given a sentence tell me whether it contains an anachronism (i.e. whether it could have happened or not based on the time periods associated with the entities).
---- 
Sentence: I wrote about shakespeare 
Entities and dates: 
 I : present 
 Shakespeare : 16th century 
Reasoning: I can write about Shakespeare because he lived in the past with respect to me. 
Anachronism: No 
---
Sentence: Shakespeare wrote about me 
Entities and dates: 
 Shakespeare : 16th century 
 I : present 
Reasoning: Shakespeare cannot have written about me, because he died before I was born 
Anachronism: Yes 
--- 
Sentence: The T-rex bit my dog 
Entities and dates:
 T-rex: 65 million years ago
My dog: present
Reasoning: The T-rex lived millions of years before my dog, so it cannot have bitten my dog.
Anachronism: Yes 
Reasoning: 
Anachronism: Yes No Yes

In [7]:
import guidance
guidance.llm = guidance.llms.OpenAI("text-davinci-003", api_key=Config.OPENAI_API_KEY)

# we can pre-define valid option sets
valid_weapons = ["sword", "axe", "mace", "spear", "bow", "crossbow"]

# define the prompt
program = guidance("""The following is a character profile for an RPG game in JSON format.
```json
{
    "description": "{{description}}",
    "name": "{{gen 'name'}}",
    "age": {{gen 'age' pattern='[0-9]+' stop=','}},
    "armor": "{{#select 'armor'}}leather{{or}}chainmail{{or}}plate{{/select}}",
    "weapon": "{{select 'weapon' options=valid_weapons}}",
    "class": "{{gen 'class'}}",
    "mantra": "{{gen 'mantra'}}",
    "strength": {{gen 'strength' pattern='[0-9]+' stop=','}},
    "items": [{{#geneach 'items' num_iterations=3}}
        "{{gen 'this'}}",{{/geneach}}
    ]
}```""")

# execute the prompt
program(description="A quick and nimble fighter.", valid_weapons=valid_weapons)

Stop program The following is a character profile for an RPG game in JSON format.
```json
{
 "description": " {{description}} ",
 "name": " {{gen 'name'}} ",
 "age": {{gen 'age' pattern='[0-9]+' stop=','}} ,
 "armor": " {{#select 'armor'}} leather {{or}} chainmail {{or}} plate {{/select}} ",
 "weapon": " {{select 'weapon' options=valid_weapons}} ",
 "class": " {{gen 'class'}} ",
 "mantra": " {{gen 'mantra'}} ",
 "strength": {{gen 'strength' pattern='[0-9]+' stop=','}} ,
 "items": [ {{#geneach 'items' num_iterations=3}} 
 " {{gen 'this'}} ", {{/geneach}} 
 ]
}```